# Подключение к tensorboared

In [1]:
# using the hard way 
# run tensorboard in kaggle server. and operate using public url

# download the files for ngrok
!wget https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
!unzip ngrok-stable-linux-amd64.zip

# Run tensorboard as well as Ngrox (for tunneling as non-blocking processes)
import os
import multiprocessing


pool = multiprocessing.Pool(processes = 10)
results_of_processes = [pool.apply_async(os.system, args=(cmd, ), callback = None )
                        for cmd in [
                        f"tensorboard --logdir ./runs/ --host 0.0.0.0 --port 6006 &",
                        "./ngrok http 6006 &"
                        ]]


--2021-07-12 09:42:41--  https://bin.equinox.io/c/4VmDzA7iaHb/ngrok-stable-linux-amd64.zip
Resolving bin.equinox.io (bin.equinox.io)... 3.212.129.206, 52.45.159.115, 3.95.52.0, ...
Connecting to bin.equinox.io (bin.equinox.io)|3.212.129.206|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 13832437 (13M) [application/octet-stream]
Saving to: ‘ngrok-stable-linux-amd64.zip’

ngrok-stable-linux- 100%[===================>]  13.19M  40.5MB/s    in 0.3s    

2021-07-12 09:42:42 (40.5 MB/s) - ‘ngrok-stable-linux-amd64.zip’ saved [13832437/13832437]

Archive:  ngrok-stable-linux-amd64.zip
  inflating: ngrok                   


In [2]:
! curl -s http://localhost:4040/api/tunnels | python3 -c \
    "import sys, json; print(json.load(sys.stdin)['tunnels'][0]['public_url'])"

http://9e982c606afc.ngrok.io


In [4]:
import numpy as np
import os, math, sys
import glob
import random

import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset, ConcatDataset
from torchvision.utils import save_image, make_grid

import matplotlib.pyplot as plt

from PIL import Image
from tqdm import tqdm_notebook as tqdm

random.seed(42)
import warnings
warnings.filterwarnings("ignore")

In [3]:
!pip install matplotlib tqdm

     |████████████████████████████████| 10.3 MB 1.4 MB/s eta 0:00:01
     |████████████████████████████████| 76 kB 6.7 MB/s  eta 0:00:01
     |████████████████████████████████| 1.2 MB 29.9 MB/s eta 0:00:01


In [5]:
# number of epochs of training
n_epochs = 50
# size of the batches
batch_size = 64
# name of the dataset
dataset_name = "val_256/"
# adam: learning rate
lr = 10**-4
# adam: decay of first order momentum of gradient
b1 = 0.5
# adam: decay of first order momentum of gradient
b2 = 0.999
# number of cpu threads to use during batch generation
n_cpu = 8
# size of each image dimension
img_size = 128
# size of random mask
mask_size = 16
# number of image channels
channels = 3

# Calculate output dims of image discriminator (PatchGAN)
patch_h, patch_w = int(mask_size / 2 ** 3), int(mask_size / 2 ** 3)
patch = (1, patch_h, patch_w)

### Define Dataset Class

In [6]:
class ImageDataset(Dataset):
    def __init__(self, root, transforms_=None, img_size=128, mask_size=16, mode="train"):
        self.transform = transforms.Compose(transforms_)
        self.img_size = img_size
        self.mask_size = mask_size
        self.mode = mode
        self.files = sorted(glob.glob("%s/*.jpg" % root))
        # self.files = self.files[:-4000] if mode == "train" else self.files[-4000:]

    def apply_random_mask(self, img):
        """Randomly masks image"""
        y1, x1 = np.random.randint(0, self.img_size - self.mask_size, 2)
        y2, x2 = y1 + self.mask_size, x1 + self.mask_size
        masked_part = img[:, y1:y2, x1:x2]
        masked_img = img.clone()
        masked_img[:, y1:y2, x1:x2] = 1

        return masked_img, masked_part

    def __getitem__(self, index):

        img = Image.open(self.files[index % len(self.files)])
        img = self.transform(img)
        if int(transforms.ToTensor()(img).shape[0]) == 1:
            img = transforms.Grayscale(num_output_channels=3)(img)
        
        img = transforms.Compose([transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),])(img)

        masked_img, aux = self.apply_random_mask(img)

        return img, masked_img, aux

    def __len__(self):
        return len(self.files)

In [7]:
transform = [
     transforms.Resize((128, 128)),
     transforms.ColorJitter(hue=.50, saturation=.50),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     ]
os.makedirs("aug_data", exist_ok=True)
augmentation_data = ImageDataset(dataset_name, transforms_=transform)

In [8]:
transform = [
     transforms.Resize((128, 128)),
     transforms.ColorJitter(hue=.20, saturation=.20),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomVerticalFlip(p=0.5),
     ]
os.makedirs("aug_data", exist_ok=True)
augmentation_data_1 = ImageDataset(dataset_name, transforms_=transform)

### Get Train/Test Dataloaders

In [9]:
transforms_ = [
    transforms.Resize((img_size, img_size), Image.BICUBIC),
    #transforms.ToTensor(),
    #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
]

dataloader = DataLoader(
    ConcatDataset([ImageDataset(dataset_name, transforms_=transforms_),augmentation_data_1, augmentation_data]),
    batch_size=batch_size,
    shuffle=True,
    num_workers=n_cpu,
)

<h3><center>Model Architecture</center></h3>
<img src="https://miro.medium.com/max/700/1*fJpamgw0yBZZRNEuex07hw.png" width="1000" height="1000"/>
<h4></h4>
<h4><center>Image Source:  <a href="https://arxiv.org/abs/1609.04802">Context Encoders: Feature Learning by Inpainting [Deepak Pathak et al.]</a></center></h4>

In [10]:
class Generator(nn.Module):
    def __init__(self, channels=3):
        super(Generator, self).__init__()

        def encoder(in_feat, out_feat, normalize=True):
            layers = [nn.Conv2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.LeakyReLU(0.2))
            return layers

        def decoder(in_feat, out_feat, normalize=True):
            layers = [nn.ConvTranspose2d(in_feat, out_feat, 4, stride=2, padding=1)]
            if normalize:
                layers.append(nn.BatchNorm2d(out_feat, 0.8))
            layers.append(nn.ReLU())
            return layers

        self.model = nn.Sequential(
            *encoder(channels, 64, normalize=False),
            *encoder(64, 64),
            *encoder(64, 128),
            *encoder(128, 256),
            *encoder(256, 512),
            nn.Conv2d(512, 4000, 1),
            *decoder(4000, 512),
            *decoder(512, 256),
            *decoder(256, 128),
            *decoder(128, 64),
            nn.Conv2d(64, channels, 3, 1, 1),
            nn.Tanh()
        )

    def forward(self, x):
        return self.model(x)
    

class Discriminator(nn.Module):
    def __init__(self, channels=3):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, stride, normalize):
            layers = [nn.Conv2d(in_filters, out_filters, 3, stride, 1)]
            if normalize:
                layers.append(nn.InstanceNorm2d(out_filters))
            layers.append(nn.LeakyReLU(0.2, inplace=True))
            return layers

        self.model = nn.Sequential(*discriminator_block(channels, 64, 2, False),
                                   *discriminator_block(64, 128, 2, True),
                                   *discriminator_block(128, 256, 2, True),
                                   *discriminator_block(256, 512, 1, True),
                                   nn.Conv2d(512, 1, 3, 1, 1))

    def forward(self, img):
        return self.model(img)

In [11]:
len(dataloader)

1711

### Train Context-Encoder GAN

In [12]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find("Conv") != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find("BatchNorm2d") != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)
    
# Loss function
adversarial_loss = torch.nn.MSELoss()
pixelwise_loss = torch.nn.L1Loss()

# Initialize generator and discriminator
generator = Generator(channels=channels)
discriminator = Discriminator(channels=channels)

# Load pretrained models
# generator.load_state_dict(torch.load("../input/context-encoder-gan-for-image-inpainting-pytorch/saved_models/generator.pth"))
# discriminator.load_state_dict(torch.load("../input/context-encoder-gan-for-image-inpainting-pytorch/saved_models/discriminator.pth"))
# print("Using pre-trained Context-Encoder GAN model!")


generator.model[27] = nn.Conv2d(64, 32, 4, stride=2, padding=1)
generator.model[28] = nn.BatchNorm2d(32, 0.8)

generator = nn.Sequential(generator,
                          nn.ReLU(),
                          nn.Conv2d(32, 16, 4, stride=2, padding=1),
                          nn.BatchNorm2d(16, 0.8),
                          nn.Conv2d(16, 3, 3, 1, 1),
                          nn.Tanh()
        )


generator.cuda()
discriminator.cuda()
adversarial_loss.cuda()
pixelwise_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

Tensor = torch.cuda.FloatTensor

In [13]:
!nvidia-smi

Mon Jul 12 21:13:58 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 470.42.01    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:04:00.0  On |                  N/A |
|  0%   55C    P8    13W / 200W |   1537MiB /  6073MiB |      4%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [15]:
!pip install tensorboard

     |████████████████████████████████| 6.0 MB 1.3 MB/s eta 0:00:01
     |████████████████████████████████| 4.9 MB 24.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.2 MB 47.1 MB/s eta 0:00:01
     |████████████████████████████████| 781 kB 14.1 MB/s eta 0:00:01
     |████████████████████████████████| 97 kB 6.8 MB/s  eta 0:00:01
     |████████████████████████████████| 132 kB 36.6 MB/s eta 0:00:01
     |████████████████████████████████| 61 kB 7.4 MB/s  eta 0:00:01
     |████████████████████████████████| 288 kB 41.8 MB/s eta 0:00:01
     |████████████████████████████████| 1.0 MB 29.8 MB/s eta 0:00:01
     |████████████████████████████████| 147 kB 26.7 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 28.4 MB/s eta 0:00:01
     |████████████████████████████████| 138 kB 34.0 MB/s eta 0:00:01
     |████████████████████████████████| 58 kB 8.4 MB/s  eta 0:00:01
     |████████████████████████████████| 145 kB 51.3 MB/s eta 0:00:01
     |████████████████████████████████

In [16]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter('runs/context_encoder')
os.makedirs("saved_models", exist_ok=True)


In [17]:
!tensorboard --logdir=runs

TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.5.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C


In [18]:

for epoch in range(n_epochs):
    
    ### Training ###
    gen_adv_loss, gen_pixel_loss, disc_loss, gen_loss = 0, 0, 0, 0
    tqdm_bar = tqdm(dataloader, desc=f'Training Epoch {epoch} ', total=int(len(dataloader)))
    for i, (imgs, masked_imgs, masked_parts) in enumerate(tqdm_bar):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], *patch).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], *patch).fill_(0.0), requires_grad=False)

        # Configure input
        imgs = Variable(imgs.type(Tensor))
        masked_imgs = Variable(masked_imgs.type(Tensor))
        masked_parts = Variable(masked_parts.type(Tensor))

        ## Train Generator ##
        optimizer_G.zero_grad()

        # Generate a batch of images
        gen_parts = generator(masked_imgs)


        # Adversarial and pixelwise loss
        g_adv = adversarial_loss(discriminator(gen_parts), valid)
        g_pixel = pixelwise_loss(gen_parts, masked_parts)
        # Total loss
        g_loss = 0.001 * g_adv + 0.999 * g_pixel

        g_loss.backward()
        optimizer_G.step()

        ## Train Discriminator ##
        optimizer_D.zero_grad()

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(masked_parts), valid)
        fake_loss = adversarial_loss(discriminator(gen_parts.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()
        

        
        gen_adv_loss += g_adv.item()
        gen_pixel_loss += g_pixel.item()
        disc_loss += d_loss.item()
        gen_loss += g_loss.item()
        tqdm_bar.set_postfix(gen_adv_loss=gen_adv_loss/(i+1), gen_pixel_loss=gen_pixel_loss/(i+1), disc_loss=disc_loss/(i+1))
        if i % 50 == 49:
            writer.add_scalar('generator loss',
                            gen_loss/(i+1),
                            epoch * len(dataloader) + i)
        
            writer.add_scalar('descremenator loss',
                            disc_loss/(i+1),
                            epoch * len(dataloader) + i)
        
         
    torch.save(generator.state_dict(), "saved_models/generator.pth")
    torch.save(discriminator.state_dict(), "saved_models/discriminator.pth")
writer.close()  

Training Epoch 0 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 1 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 2 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 3 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 4 :   0%|          | 0/1711 [00:00<?, ?it/s]

...

Training Epoch 45 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 46 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 47 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 48 :   0%|          | 0/1711 [00:00<?, ?it/s]

Training Epoch 49 :   0%|          | 0/1711 [00:00<?, ?it/s]